In [ ]:
# # Install required libraries
#%pip install matplotlib pandas numpy seaborn plotly

In [ ]:
!matplotlib notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime
import os
import warnings
import plotly.express as px
import plotly.io as pio

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


# pio.renderers.default = "colab"
pio.renderers.default = "vscode"      # <- the VS Code renderer
# pio.renderers.default = "notebook"  # works too; loads plotly.js inline
# pio.renderers.default = "iframe"    # good for static HTML export

# Set plot styling and suppress warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['axes.grid'] = True
plt.rcParams['font.size'] = 12

## 2. Data Loading and Preparation

Load the sensor data from CSV, convert timestamps to datetime format, and prepare the data for time-series analysis.

In [ ]:
# # Define the data file path
# file_path = os.path.join('data', 'sensor_readings_export_20250426_132445.csv')

# Load the data
df0 = pd.read_csv('../raw/sensor_readings_export_20250501_184851.csv')

# Display the first few rows to understand the data structure
print(f"Data shape: {df0.shape}")
df0.head()

In [ ]:
# Check unique device IDs in the dataset
unique_devices = df0['device_id'].unique()
print(f"Unique device IDs in the dataset: {unique_devices}")

In [ ]:
# Filter the dataset to only include records from device_id 'esp32_01'
df0 = df0[df0['device_id'] == 'esp32_01']

# Display first few rows of the filtered dataset
df0.head()

### 2.1 Data Cleaning and Preprocessing

In [ ]:
# Check data types
print("\nData types:")
print(df0.dtypes)

In [ ]:
# Check for missing values
print("Missing values per column:")
print(df0.isnull().sum())

In [ ]:
# Drop specified columns
df1 = df0.drop(['smoke_detected', 'wildfire_detected', 'thresholds_exceeded', 'potential_wildfire', 'device_id', '_id'], axis=1)

# Display the updated DataFrame info
print("Updated DataFrame columns:")
print(df1.columns.tolist())

In [ ]:
df1.head()

In [ ]:
# Check for missing values
print("Missing values per column:")
print(df1.isnull().sum())

In [ ]:
# Create a clean copy of the data for our analysis
df2 = df1.copy()

# Remove rows where ir_temperature is missing
df3 = df2.dropna(subset=['ir_temperature'])

# Check the shape after removing rows with missing ir_temperature
print(f"Data shape after removing rows with missing ir_temperature: {df3.shape}")

# Check for missing values in the remaining data
print("\nMissing values after cleaning:")
print(df3.isnull().sum())

In [ ]:
# Display the cleaned data
print("After cleaning:")
print(f"Data shape: {df3.shape}")

# Display a few rows of the cleaned data
df3.head()

In [ ]:
# Convert timestamp to datetime (force UTC to handle tz‑aware strings)
df3['timestamp'] = pd.to_datetime(df3['timestamp'], utc=True)

# (Optional) Drop the timezone info to get naive datetimes
df3['timestamp'] = df3['timestamp'].dt.tz_convert(None)

# Now filter years > 2025
df3 = df3[df3['timestamp'].dt.year >= 2025]

In [ ]:
df3

In [ ]:
df3.shape

In [ ]:
# Set timestamp as index for time-series analysis
df3.set_index('timestamp', inplace=True)

# Convert numerical columns to float
numerical_columns = ['temperature', 'humidity', 'smoke', 'ir_temperature']
for col in numerical_columns:
    if col in df3.columns:
        df3[col] = pd.to_numeric(df3[col], errors='coerce')

# Handle missing values by filling with column means
df3[numerical_columns] = df3[numerical_columns].fillna(df3[numerical_columns].mean())

# Display the cleaned data
print("After cleaning:")
print(f"Data shape: {df3.shape}")
df3.head()

### 2.2 Removing Data Points Based on Thresholds

Let's remove data points that fall below certain thresholds which may indicate sensor errors or irrelevant conditions for wildfire analysis:

In [ ]:
# Display shape before filtering
print(f"Data shape before filtering: {df3.shape}")

# Store original DataFrame for reference
df_original = df3.copy()

# Filter out rows with values below thresholds
filtered_df = df3[
    (df3['temperature'] > 30) &
    (df3['humidity'] > 10) &
    (df3['smoke'] > 1000) &
    (df3['ir_temperature'] > 20)
]

# Reassign filtered DataFrame to df3 for subsequent analysis
df4 = filtered_df.copy()

# Display the results of filtering
print(f"Data shape after filtering: {df4.shape}")
print(f"Removed {df3.shape[0] - df4.shape[0]} rows based on threshold conditions")

# Display percentage of data retained
retention_percentage = (df4.shape[0] / df3.shape[0]) * 100
print(f"Retained {retention_percentage:.2f}% of the original data after filtering")

# Display first few rows of filtered data
df4.head()

In [ ]:
# Filter data for dates after April 25, 2025
df4 = df4[(df4.index >= '2025-04-25')]
df4.shape

### 2.3 Data Processing Flow

For clarity, here's a summary of our data processing pipeline:

1. **Initial Load**: Loaded raw sensor data from CSV file
2. **Device Filtering**: Selected only data from device 'esp32_01'
3. **Column Reduction**: Removed non-essential columns
4. **Missing Value Handling**: Removed rows with missing IR temperature values
5. **Timestamp Conversion**: Converted timestamps to datetime format
6. **Normalization**: Ensured numerical data types for sensor readings
7. **Threshold Filtering**: Applied minimum thresholds to filter out irrelevant data

This clean, time-indexed dataset (df4) will be the foundation for all subsequent analyses.

## 2.4 Data Validation and Outlier Detection

Before proceeding with analysis, we need to validate our data and identify any anomalous values that could skew our results. We'll use multiple methods to detect outliers in our sensor readings.

In [ ]:
# Import required libraries for outlier detection visualizations
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# Let's check the basic ranges of our data first
print("Data range by sensor:")
for column in numerical_columns:
    min_val = df4[column].min()
    max_val = df4[column].max()
    mean_val = df4[column].mean()
    std_val = df4[column].std()
    print(f"{column}: Min={min_val:.2f}, Max={max_val:.2f}, Mean={mean_val:.2f}, Std={std_val:.2f}")

### 2.4.1 Z-Score Based Outlier Detection

The Z-score method identifies outliers by measuring how many standard deviations a data point is from the mean. Values with a Z-score greater than 3 or less than -3 are typically considered outliers.

In [ ]:
def detect_outliers_zscore(df, columns, threshold=3):
    """Detect outliers using the Z-score method
    
    Args:
        df: DataFrame containing the data
        columns: List of column names to check for outliers
        threshold: Z-score threshold (default: 3)
        
    Returns:
        DataFrame with outlier flags and Z-scores
    """
    outlier_df = df.copy()
    
    for column in columns:
        # Calculate Z-scores
        z_scores = (df[column] - df[column].mean()) / df[column].std()
        
        # Add Z-scores to dataframe
        outlier_df[f'{column}_zscore'] = z_scores
        
        # Flag outliers
        outlier_df[f'{column}_outlier'] = (abs(z_scores) > threshold)
        
    return outlier_df

# Apply Z-score outlier detection
outliers_zscore = detect_outliers_zscore(df4, numerical_columns)

# Count outliers detected by Z-score method
print("Outliers detected using Z-score method:")
for column in numerical_columns:
    outlier_count = outliers_zscore[f'{column}_outlier'].sum()
    outlier_percent = (outlier_count / len(outliers_zscore)) * 100
    print(f"{column}: {outlier_count} outliers ({outlier_percent:.2f}% of data)")

### 2.4.2 IQR Based Outlier Detection

The Interquartile Range (IQR) method is robust to non-normal distributions. It identifies outliers as values that fall below Q1 - 1.5*IQR or above Q3 + 1.5*IQR.

In [ ]:
def detect_outliers_iqr(df, columns, factor=1.5):
    """Detect outliers using the IQR method
    
    Args:
        df: DataFrame containing the data
        columns: List of column names to check for outliers
        factor: IQR multiplier for determining outliers (default: 1.5)
        
    Returns:
        DataFrame with outlier flags and bounds
    """
    outlier_df = df.copy()
    
    for column in columns:
        # Calculate IQR
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        
        # Calculate bounds
        lower_bound = Q1 - factor * IQR
        upper_bound = Q3 + factor * IQR
        
        # Add bounds to dataframe
        outlier_df[f'{column}_lower_bound'] = lower_bound
        outlier_df[f'{column}_upper_bound'] = upper_bound
        
        # Flag outliers
        outlier_df[f'{column}_outlier'] = ((df[column] < lower_bound) | (df[column] > upper_bound))
        
    return outlier_df

# Apply IQR outlier detection
outliers_iqr = detect_outliers_iqr(df4, numerical_columns)

# Count outliers detected by IQR method
print("Outliers detected using IQR method:")
for column in numerical_columns:
    outlier_count = outliers_iqr[f'{column}_outlier'].sum()
    outlier_percent = (outlier_count / len(outliers_iqr)) * 100
    print(f"{column}: {outlier_count} outliers ({outlier_percent:.2f}% of data)")

### 2.4.3 Visualize Outliers

Let's visualize the outliers using box plots and scatter plots to understand their distribution.

In [ ]:
# Create box plots to visualize outliers
fig = make_subplots(
    rows=2, 
    cols=2,
    subplot_titles=numerical_columns
)

row, col = 1, 1
for column in numerical_columns:
    fig.add_trace(
        go.Box(
            y=df4[column],
            name=column,
            boxpoints='outliers',  # Only show outliers
            marker_color='rgb(8,81,156)',
            boxmean=True  # Show mean
        ),
        row=row, col=col
    )
    
    # Update position for next plot
    col += 1
    if col > 2:
        col = 1
        row += 1

# Update layout
fig.update_layout(
    title_text="Box plots showing outliers in sensor data",
    height=700,
    width=1000
)

fig.show()

In [ ]:
# Reset index to use timestamp as a column
plot_df = df4.reset_index()

# Create a scatter plot showing outliers over time for each sensor
for column in numerical_columns:
    # Merge the main dataframe with outlier flags
    temp_df = plot_df.copy()
    temp_df['outlier'] = outliers_iqr[f'{column}_outlier'].values
    
    # Create figure
    plt.figure(figsize=(14, 6))
    
    # Plot normal data points
    sns.scatterplot(
        data=temp_df[~temp_df['outlier']],
        x='timestamp',
        y=column,
        color='blue',
        s=20,
        alpha=0.6,
        label='Normal Data'
    )
    
    # Plot outlier data points
    sns.scatterplot(
        data=temp_df[temp_df['outlier']],
        x='timestamp',
        y=column,
        color='red',
        s=50,
        marker='X',
        label='Outliers'
    )
    
    # Configure plot
    plt.title(f'Outliers in {column} over time', fontsize=16)
    plt.xlabel('Time', fontsize=12)
    plt.ylabel(column, fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

### 2.4.4 Handle Outliers

Now we'll handle the detected outliers using different methods:
1. Remove outliers
2. Cap outliers at the thresholds
3. Replace outliers with interpolated values

In [ ]:
def handle_outliers(df, method='cap', columns=None, outlier_flags=None):
    """Handle outliers using various methods
    
    Args:
        df: DataFrame containing the data
        method: Method to handle outliers ('remove', 'cap', 'interpolate')
        columns: List of column names with outliers to handle
        outlier_flags: DataFrame with outlier flags
        
    Returns:
        DataFrame with handled outliers
    """
    result_df = df.copy()
    
    if columns is None:
        columns = df.select_dtypes(include='number').columns.tolist()
    
    if method == 'remove':
        # Create mask for rows to keep (those without outliers in any column)
        mask = pd.Series(False, index=df.index)
        for column in columns:
            mask = mask | outlier_flags[f'{column}_outlier']
        
        # Keep only non-outlier rows
        result_df = df[~mask]
        
    elif method == 'cap':
        # Cap outliers at the IQR bounds
        for column in columns:
            lower_bound = outlier_flags[f'{column}_lower_bound'].iloc[0]
            upper_bound = outlier_flags[f'{column}_upper_bound'].iloc[0]
            
            # Cap values that are too low
            mask_low = df[column] < lower_bound
            result_df.loc[mask_low, column] = lower_bound
            
            # Cap values that are too high
            mask_high = df[column] > upper_bound
            result_df.loc[mask_high, column] = upper_bound
            
    elif method == 'interpolate':
        # Replace outliers with interpolated values
        for column in columns:
            # Create a copy of the column
            temp_values = df[column].copy()
            
            # Set outliers to NaN
            mask = outlier_flags[f'{column}_outlier']
            temp_values[mask] = np.nan
            
            # Interpolate NaN values
            result_df[column] = temp_values.interpolate(method='time').fillna(
                method='bfill').fillna(method='ffill')
            
    return result_df

# Let's apply all three methods and compare results
df_no_outliers_removed = handle_outliers(df4, method='remove', columns=numerical_columns, outlier_flags=outliers_iqr)
df_no_outliers_capped = handle_outliers(df4, method='cap', columns=numerical_columns, outlier_flags=outliers_iqr)
df_no_outliers_interpolated = handle_outliers(df4, method='interpolate', columns=numerical_columns, outlier_flags=outliers_iqr)

# Show the results
print(f"Original data shape: {df4.shape}")
print(f"Shape after removing outliers: {df_no_outliers_removed.shape}")
print(f"Shape after capping outliers: {df_no_outliers_capped.shape} (same as original)")
print(f"Shape after interpolating outliers: {df_no_outliers_interpolated.shape} (same as original)")

### 2.4.5 Choose Outlier Handling Method

Based on the exploration above, let's select the most appropriate outlier handling method for our analysis. The capping method preserves the time series structure while limiting extreme values.

In [ ]:
# Let's use the capped version for our further analysis
df4_cleaned = df_no_outliers_capped.copy()

# Compare statistical summary before and after outlier handling
print("Statistics before outlier handling:")
print(df4[numerical_columns].describe().round(2))

print("\nStatistics after outlier handling:")
print(df4_cleaned[numerical_columns].describe().round(2))

# Use this cleaned dataframe for further analysis
df4 = df4_cleaned

In [ ]:
df4.shape

### 2.4.6 Validate Data Consistency

Let's perform additional validation checks to ensure data consistency after cleaning.

In [ ]:
# # Check for physical consistency across sensors
# # Example: In wildfires, temperature and smoke should be positively correlated

# # Check correlations between sensor readings
# correlations = df4[numerical_columns].corr()

# # Create correlation heatmap
# fig = px.imshow(
#     correlations,
#     text_auto=True,
#     color_continuous_scale='RdBu_r',
#     title='Correlation Between Sensors After Cleaning'
# )
# fig.show()

# # Check for physical inconsistencies (e.g., temperature dropping while smoke increases)
# print("\nChecking for physical inconsistencies...")
# temp_smoke_corr = correlations.loc['temperature', 'smoke']
# print(f"Correlation between temperature and smoke: {temp_smoke_corr:.4f}")

# if temp_smoke_corr < 0.3:
#     print("WARNING: Low correlation between temperature and smoke might indicate sensor issues")
# else:
#     print("Sensor readings show expected physical correlations")


# Calculate correlation matrix
correlation_matrix = df4[numerical_columns].corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt='.2f')
plt.title('Correlation Matrix Between Sensor Readings')
plt.tight_layout()
plt.show()

## 3. Basic Statistical Summary

Let's examine the basic statistics of our sensor data to understand the distribution.

In [ ]:
# Generate descriptive statistics
df4[numerical_columns].describe()

## 4. Temporal Pattern Analysis

In this section, we'll analyze how sensor readings change over time, looking at both daily patterns and longer-term trends. This will help identify periods of higher wildfire risk and understand the environmental factors that contribute to those risks.

### 4.1 Sensor Data Time Series Overview

First, let's visualize the overall trends in our key sensor readings across the entire dataset period.

### 4.2 Daily Patterns Analysis

Let's extract the hour of day to analyze daily patterns.

In [ ]:
# Filter data for dates after April 25, 2025
df5 = df4.copy()

# Display basic information about the filtered dataset
print(f"Original data shape: {df4.shape}")
print(f"Filtered data shape: {df5.shape}")
print("\nDate range in filtered data:")
print(f"Start: {df5.index.min()}")
print(f"End: {df5.index.max()}")

In [ ]:
df5

In [ ]:
# Resample data to 1-minute intervals to reduce noise while maintaining temporal detail
# This provides a good balance for time series visualization and pattern recognition
df5_downsampled = df5.resample('1T').mean()

# Display basic info about the resampled data
print(f"Original data points: {df5.shape[0]}")
print(f"Resampled data points: {df5_downsampled.shape[0]}")
print(f"Reduction ratio: {df5_downsampled.shape[0]/df5.shape[0]:.2%}")

### 4.2.1 Individual Sensor Visualizations

Below we plot each sensor's readings over the course of April 25, 2025. These visualizations help identify how different environmental factors change throughout the day and may reveal patterns related to wildfire risk.

In [ ]:
# 3. Plot
plt.figure(figsize=(14, 6))
ax = sns.lineplot(
    x=df5_downsampled.index,
    y=df5_downsampled['temperature'],
    color='orangered',
    linewidth=1
)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(mdates.HourLocator(interval=5))
plt.title('Temperature Readings Over Time')
plt.xlabel('Hour of Day')
plt.ylabel('Temperature (°C)')
plt.tight_layout()
plt.show()


In [ ]:
fig = px.line(
    df5_downsampled,
    x=df5_downsampled.index,
    y="temperature",
    labels={"temperature": "Temperature (°C)", "index": "Time"},
    title="Temperature Readings Over Time"
)
fig.show()

In [ ]:
fig = px.line(
    df5_downsampled,
    x=df5_downsampled.index,
    y="smoke",
    labels={"smoke": "Smoke (°C)", "index": "Time"},
    title="Smoke Readings Over Time"
)
fig.show()

In [ ]:
fig = px.line(
    df5_downsampled,
    x=df5_downsampled.index,
    y="humidity",
    labels={"humidity": "Humidity (°C)", "index": "Time"},
    title="Humidity Readings Over Time"
)
fig.show()

In [ ]:
fig = px.line(
    df5_downsampled,
    x=df5_downsampled.index,
    y="ir_temperature",
    labels={"ir_temperature": "Inferred Temperature (°C)", "index": "Time"},
    title="Inferred Temperature Readings Over Time"
)
fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create a 2x2 grid of subplots
fig = make_subplots(
    rows=4,
    cols=1,
    subplot_titles=(
        "Temperature Readings",
        "IR Temperature Readings"
        "Humidity Readings",
        "Smoke Readings",
    )
)

# Add traces for each sensor
fig.add_trace(
    go.Scatter(
        x=df5_downsampled.index,
        y=df5_downsampled["temperature"],
        name="Temperature"
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df5_downsampled.index,
        y=df5_downsampled["ir_temperature"],
        name="IR Temperature"
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=df5_downsampled.index,
        y=df5_downsampled["humidity"],
        name="Humidity"
    ),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(
        x=df5_downsampled.index,
        y=df5_downsampled["smoke"],
        name="Smoke"
    ),
    row=4, col=1
)



# Update layout
fig.update_layout(
    height=800,
    width=1000,
    title_text="Sensor Readings Over Time (April 25, 2025)",
    showlegend=True
)

# Update y-axis labels
fig.update_yaxes(title_text="Temperature (°C)", row=1, col=1)
fig.update_yaxes(title_text="IR Temperature (°C)", row=2, col=1)
fig.update_yaxes(title_text="Humidity (%)", row=3, col=1)
fig.update_yaxes(title_text="Smoke Level", row=4, col=1)

fig.show()

### 4.3 Daily Aggregations

Resample the data to see daily patterns and trends.

## 5. Correlation Analysis Between Sensors

Analyze how different sensors relate to each other over time.

### 5.1 Understanding Sensor Correlations

The correlation heatmap below provides insights into how different environmental factors interact with each other in our monitoring system:

- **Positive correlations** indicate sensors that tend to increase together
- **Negative correlations** indicate that when one sensor reading increases, the other tends to decrease
- **Strong correlations** (closer to 1 or -1) indicate a more reliable relationship between sensor readings

These relationships are particularly valuable for identifying key indicators and potential redundancies in our wildfire detection system.

In [ ]:
# Calculate correlation matrix
correlation_matrix = df4[numerical_columns].corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt='.2f')
plt.title('Correlation Matrix Between Sensor Readings')
plt.tight_layout()
plt.show()

## 6. Feature Engineering for Enhanced Wildfire Detection

Feature engineering is crucial for improving model performance in time series forecasting and anomaly detection tasks. In this section, we'll create several types of features that can help our models better capture the complex patterns and relationships in our environmental sensor data:

1. **Time-based features**: Extract cyclical temporal patterns
2. **Rolling window features**: Capture trends and variability over different time horizons
3. **Lag features**: Provide historical context to identify deviations
4. **Domain-specific features**: Create combinations that have physical meaning in wildfire contexts
5. **Feature importance analysis**: Determine which engineered features contribute most to predictions
6. **Feature selection**: Create optimized feature sets for model training

These engineered features will help our models detect early warning signs of potential wildfire conditions.

In [ ]:
# Reset index to get timestamp as a column for easier feature engineering
df_fe = df5_downsampled.reset_index().copy()

# Check the structure of our dataset
print(f"Dataset shape: {df_fe.shape}")
df_fe.head()

### 6.1 Time-based Feature Engineering

Time-based features help capture temporal patterns and cyclical behaviors in the data. We'll extract components like hour of day, day of week, etc., and use cyclical encoding to preserve their circular nature.

In [ ]:
import math

def cyclical_encoding(df, col, period):
    """Create cyclical encoding for temporal features to preserve their circular nature"""
    df[f'{col}_sin'] = np.sin(2 * np.pi * df[col]/period)
    df[f'{col}_cos'] = np.cos(2 * np.pi * df[col]/period)
    return df

# Extract time components
df_fe['hour'] = df_fe['timestamp'].dt.hour
df_fe['day'] = df_fe['timestamp'].dt.day
df_fe['day_of_week'] = df_fe['timestamp'].dt.dayofweek
df_fe['month'] = df_fe['timestamp'].dt.month
df_fe['is_day'] = ((df_fe['hour'] >= 6) & (df_fe['hour'] <= 18)).astype(int)

# Apply cyclical encoding to preserve circular nature of time features
df_fe = cyclical_encoding(df_fe, 'hour', 24)
df_fe = cyclical_encoding(df_fe, 'day_of_week', 7)
df_fe = cyclical_encoding(df_fe, 'month', 12)
df_fe = cyclical_encoding(df_fe, 'day', 31)

# Display results
time_cols = ['hour', 'hour_sin', 'hour_cos', 'day_of_week', 'day_of_week_sin', 'day_of_week_cos', 'is_day']
df_fe[['timestamp'] + time_cols].head()

In [ ]:
# Visualize the cyclical encoding of hour of day
plt.figure(figsize=(10, 6))

# Create a full 24-hour circle for visualization
hours = np.arange(0, 24, 1)
hours_sin = np.sin(2 * np.pi * hours/24)
hours_cos = np.cos(2 * np.pi * hours/24)

plt.scatter(hours_cos, hours_sin, c=hours, cmap='hsv', s=100, alpha=0.8)
for i, hour in enumerate(hours):
    plt.annotate(f"{hour}", (hours_cos[i], hours_sin[i]), xytext=(5, 5), textcoords='offset points')
    
plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
plt.title('Cyclical Encoding of Hour of Day')
plt.xlabel('Cosine Component')
plt.ylabel('Sine Component')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()

### 6.2 Rolling Window Features

Rolling window features capture trends and variability over different time horizons. We'll compute statistics over various window sizes to detect anomalies and changes in sensor readings.

In [ ]:
# First ensure data is sorted by timestamp
df_fe = df_fe.sort_values('timestamp')

# Define window sizes in minutes
window_sizes = [10, 30, 60]  # 10min, 30min, 1hour

# Set timestamp as index temporarily for rolling operations
df_rolling = df_fe.set_index('timestamp')

# Generate rolling features for each sensor and window size
for col in numerical_columns:
    for window in window_sizes:
        # Calculate rolling statistics
        df_rolling[f'{col}_rolling_mean_{window}m'] = df_rolling[col].rolling(f'{window}min').mean()
        df_rolling[f'{col}_rolling_std_{window}m'] = df_rolling[col].rolling(f'{window}min').std()
        df_rolling[f'{col}_rolling_max_{window}m'] = df_rolling[col].rolling(f'{window}min').max()
        df_rolling[f'{col}_rolling_min_{window}m'] = df_rolling[col].rolling(f'{window}min').min()
        
        # Calculate rate of change over window (first derivative)
        df_rolling[f'{col}_rate_change_{window}m'] = df_rolling[col].diff(window)
        
        # Calculate acceleration (second derivative) - change in rate of change
        df_rolling[f'{col}_acceleration_{window}m'] = df_rolling[f'{col}_rate_change_{window}m'].diff(window)

# Reset index to get timestamp back as column
df_fe = df_rolling.reset_index()

# Show examples of rolling features
cols_to_show = ['timestamp', 'temperature', 'temperature_rolling_mean_60m', 'temperature_rate_change_60m']
df_fe[cols_to_show].dropna().head()

In [ ]:
# Visualize temperature with its rolling mean and standard deviation
plt.figure(figsize=(14, 7))

# Sample a shorter time period for clearer visualization
sample_period = df_fe[(df_fe['timestamp'] >= '2025-04-26') & (df_fe['timestamp'] < '2025-04-27')]

plt.plot(sample_period['timestamp'], sample_period['temperature'], 'b-', label='Temperature', alpha=0.7)
plt.plot(sample_period['timestamp'], sample_period['temperature_rolling_mean_60m'], 'r-', 
         label='1-hour Rolling Mean', linewidth=2)

# Add rolling standard deviation bands
plt.fill_between(sample_period['timestamp'],
                 sample_period['temperature_rolling_mean_60m'] - sample_period['temperature_rolling_std_60m'],
                 sample_period['temperature_rolling_mean_60m'] + sample_period['temperature_rolling_std_60m'],
                 color='gray', alpha=0.2, label='±1 Std Dev')

plt.xlabel('Time')
plt.ylabel('Temperature (°C)')
plt.title('Temperature with 1-hour Rolling Statistics (April 26, 2025)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 6.3 Lag Features

Lag features provide historical context that helps models detect deviations from expected patterns. We'll create lagged values at different time intervals.

In [ ]:
# Define lag periods in minutes
lag_periods = [10, 30, 60, 120]  # 10min, 30min, 1hour, 2hours

# Set timestamp as index temporarily for shift operations
df_lagged = df_fe.set_index('timestamp')

# Generate lag features for each sensor
for col in numerical_columns:
    for lag in lag_periods:
        # Create lagged values
        df_lagged[f'{col}_lag_{lag}m'] = df_lagged[col].shift(freq=f'{lag}min')
        
        # Calculate difference from lagged value (change over time)
        df_lagged[f'{col}_diff_{lag}m'] = df_lagged[col] - df_lagged[f'{col}_lag_{lag}m']
        
        # Calculate percentage change
        df_lagged[f'{col}_pct_change_{lag}m'] = df_lagged[col].pct_change(freq=f'{lag}min')

# Reset index to get timestamp back as column
df_fe = df_lagged.reset_index()

# Show examples of lag features
cols_to_show = ['timestamp', 'temperature', 'temperature_lag_60m', 'temperature_diff_60m', 'temperature_pct_change_60m']
df_fe[cols_to_show].dropna().head()

In [ ]:
# Visualize temperature with its 1-hour lag
plt.figure(figsize=(14, 7))

# Sample a shorter time period for clearer visualization
sample = df_fe[(df_fe['timestamp'] >= '2025-04-27') & (df_fe['timestamp'] < '2025-04-28')].dropna(subset=['temperature_lag_60m'])

plt.plot(sample['timestamp'], sample['temperature'], 'b-', label='Temperature', linewidth=2)
plt.plot(sample['timestamp'], sample['temperature_lag_60m'], 'g--', label='Temperature (1 hour ago)', linewidth=2)
plt.plot(sample['timestamp'], sample['temperature_diff_60m'], 'r-', label='1-hour Difference', linewidth=1.5)

plt.xlabel('Time')
plt.ylabel('Temperature (°C)')
plt.title('Temperature with 1-hour Lag Features (April 27, 2025)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 6.4 Domain-Specific Features for Wildfire Detection

We'll create combinations of existing features that have physical meaning in the context of wildfire detection, such as heat-humidity index, dryness factor, rapid temperature rise indicators, etc.

In [ ]:
# Fill NaN values with median for feature calculations
df_fe = df_fe.fillna(df_fe.median())

# Create domain-specific features for wildfire detection

# 1. Heat-Dryness Index (higher values indicate higher fire risk)
# Combines temperature and inverse of humidity
df_fe['heat_dryness_index'] = df_fe['temperature'] * (100 - df_fe['humidity']) / 100

# 2. Temperature-IR Differential (can indicate hidden heat sources)
df_fe['temp_ir_differential'] = df_fe['ir_temperature'] - df_fe['temperature']

# 3. Smoke-Temperature Ratio (smoke concentration relative to temperature)
df_fe['smoke_temp_ratio'] = df_fe['smoke'] / df_fe['temperature']

# 4. Rapid Temperature Rise Indicator
# Using 30-minute difference, normalized by standard deviation
temp_std = df_fe['temperature_rolling_std_30m'].replace(0, 0.1)  # Avoid division by zero
df_fe['rapid_temp_rise'] = df_fe['temperature_diff_30m'] / temp_std

# 5. Fire Danger Index - custom formula combining multiple factors
# Higher temperature, lower humidity, higher smoke, higher IR temperature all contribute to higher index
df_fe['fire_danger_index'] = (
    df_fe['temperature'] * 0.3 + 
    (100 - df_fe['humidity']) * 0.3 + 
    df_fe['smoke'] / 1000 * 0.2 + 
    df_fe['ir_temperature'] * 0.2
)

# 6. Rate of Change Composite (combines rate of change for multiple sensors)
df_fe['multi_sensor_rate'] = (
    df_fe['temperature_rate_change_30m'] * 0.4 + 
    df_fe['smoke_rate_change_30m'] * 0.4 + 
    df_fe['ir_temperature_rate_change_30m'] * 0.2
)

# Display the new domain-specific features
domain_features = [
    'heat_dryness_index', 'temp_ir_differential', 'smoke_temp_ratio',
    'rapid_temp_rise', 'fire_danger_index', 'multi_sensor_rate'
]

df_fe[['timestamp'] + domain_features].head()

In [ ]:
# Visualize the fire danger index over time
plt.figure(figsize=(14, 7))

# Create a time subset for clearer visualization
time_subset = df_fe[(df_fe['timestamp'] >= '2025-04-28') & (df_fe['timestamp'] < '2025-04-30')]

plt.plot(time_subset['timestamp'], time_subset['fire_danger_index'], 'r-', label='Fire Danger Index', linewidth=2)

# Add a reference line for a moderate risk threshold
moderate_risk = np.percentile(df_fe['fire_danger_index'], 75)  # 75th percentile as threshold
high_risk = np.percentile(df_fe['fire_danger_index'], 90)      # 90th percentile as threshold

plt.axhline(y=moderate_risk, color='orange', linestyle='--', label=f'Moderate Risk Threshold ({moderate_risk:.2f})')
plt.axhline(y=high_risk, color='darkred', linestyle='--', label=f'High Risk Threshold ({high_risk:.2f})')

plt.fill_between(time_subset['timestamp'], moderate_risk, high_risk, color='orange', alpha=0.2)
plt.fill_between(time_subset['timestamp'], high_risk, max(time_subset['fire_danger_index'])*1.1, color='red', alpha=0.2)

plt.xlabel('Time')
plt.ylabel('Fire Danger Index')
plt.title('Fire Danger Index Over Time (April 28-29, 2025)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 6.5 Feature Importance Analysis

We'll use machine learning models to identify which of our engineered features contribute most to predicting each target variable.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt

# Function to analyze feature importance for a target variable
def analyze_feature_importance(df, target_col, top_n=20):
    # Drop columns with any NaN values for this analysis
    df_clean = df.dropna(axis=1, how='any')
    
    # Drop the timestamp column and other targets
    feature_cols = [col for col in df_clean.columns if col != 'timestamp' and col != target_col 
                    and col not in [c for c in numerical_columns if c != target_col]]
    
    # Prepare feature matrix and target vector
    X = df_clean[feature_cols]
    y = df_clean[target_col]
    
    print(f"Analyzing feature importance for predicting {target_col}...")
    print(f"Number of features: {len(feature_cols)}")
    print(f"Number of samples: {len(X)}")
    
    # Train a Random Forest model
    model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X, y)
    
    # Get feature importances
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]  # Sort in descending order
    
    # Select top N features for visualization
    top_indices = indices[:top_n]
    top_features = [feature_cols[i] for i in top_indices]
    top_importances = importances[top_indices]
    
    # Calculate cumulative importance
    cumulative_importance = np.cumsum(importances[indices])
    num_features_90pct = np.where(cumulative_importance >= 0.9)[0][0] + 1
    
    print(f"Number of features needed for 90% of predictive power: {num_features_90pct}")
    
    # Visualize feature importance
    plt.figure(figsize=(10, 12))
    plt.barh(range(len(top_importances)), top_importances, color="skyblue")
    plt.yticks(range(len(top_importances)), [top_features[i] for i in range(len(top_importances))])
    plt.xlabel('Feature Importance')
    plt.ylabel('Feature')
    plt.title(f'Top {top_n} Most Important Features for Predicting {target_col}')
    plt.tight_layout()
    plt.show()
    
    # Return top features and their importance scores
    return dict(zip(top_features, top_importances))

In [ ]:
# Analyze feature importance for each target variable
importance_results = {}

for target in numerical_columns:
    importance_results[target] = analyze_feature_importance(df_fe, target, top_n=15)

### 6.6 Feature Selection

Based on the feature importance analysis, we'll create optimized datasets for each target variable with only the most predictive features.

In [ ]:
from sklearn.feature_selection import SelectFromModel

def select_important_features(df, target_col, importance_threshold=0.01):
    """Select features with importance above the specified threshold"""
    # Drop columns with any NaN values
    df_clean = df.dropna(axis=1, how='any')
    
    # Drop the timestamp column and other targets for feature selection
    feature_cols = [col for col in df_clean.columns if col != 'timestamp' and col != target_col 
                    and col not in [c for c in numerical_columns if c != target_col]]
    
    # Prepare feature matrix and target vector
    X = df_clean[feature_cols]
    y = df_clean[target_col]
    
    # Train a Random Forest model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)
    
    # Select features using the model's feature importances
    selector = SelectFromModel(model, threshold=importance_threshold, prefit=True)
    feature_mask = selector.get_support()
    selected_features = [feature for feature, selected in zip(feature_cols, feature_mask) if selected]
    
    print(f"Original number of features: {len(feature_cols)}")
    print(f"Selected number of features for {target_col}: {len(selected_features)}")
    
    # Create a dataset with only the selected features and the target
    selected_cols = ['timestamp'] + selected_features + [target_col]
    selected_df = df[selected_cols].copy()
    
    return selected_df, selected_features

# Create optimized datasets for each target
optimized_datasets = {}
selected_features_dict = {}

for target in numerical_columns:
    print(f"\nSelecting features for {target}:")
    optimized_datasets[target], selected_features_dict[target] = select_important_features(df_fe, target, importance_threshold=0.01)
    
    # Show the first few rows of the optimized dataset
    print(f"\nOptimized dataset for {target} (first few rows):")
    display(optimized_datasets[target].head(3))
    print(f"Shape: {optimized_datasets[target].shape}")

### 6.7 Prepare Final Feature Engineered Datasets for Modeling

Now we'll prepare the final datasets with the selected features for each target variable, ready for model training.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Create a function to prepare a dataset for modeling
def prepare_for_modeling(df, target_col, sequence_length=60):
    """Prepare an engineered dataset for time series modeling"""
    # Drop rows with NaN values
    df_clean = df.dropna().copy()
    
    # Set timestamp as index
    df_clean.set_index('timestamp', inplace=True)
    
    # Get feature columns (excluding the target)
    feature_cols = [col for col in df_clean.columns if col != target_col]
    
    # Scale the features and target
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    
    # Scale the features
    if feature_cols:  # Check if there are any features
        df_clean[feature_cols] = scaler_x.fit_transform(df_clean[feature_cols])
    
    # Scale the target
    df_clean[[target_col]] = scaler_y.fit_transform(df_clean[[target_col]])
    
    # Create sequences for LSTM
    X, y = [], []
    for i in range(len(df_clean) - sequence_length):
        # Create a sequence of all features
        X_seq = df_clean.iloc[i:(i + sequence_length)].values
        # Get the next value of the target
        y_seq = df_clean.iloc[i + sequence_length][target_col]
        X.append(X_seq)
        y.append(y_seq)
    
    X, y = np.array(X), np.array(y)
    
    # Split into training and testing sets (80% train, 20% test)
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]
    
    print(f"Prepared dataset for {target_col}:")
    print(f"  X_train shape: {X_train.shape}")
    print(f"  X_test shape: {X_test.shape}")
    
    return {
        'X_train': X_train, 
        'X_test': X_test, 
        'y_train': y_train, 
        'y_test': y_test,
        'scaler_x': scaler_x,
        'scaler_y': scaler_y,
        'feature_cols': feature_cols,
        'target_col': target_col
    }

# Prepare model-ready datasets for each target
model_datasets = {}

for target in numerical_columns:
    print(f"\nPreparing model dataset for {target}...")
    model_datasets[target] = prepare_for_modeling(optimized_datasets[target], target)

### 6.8 Feature Engineering Summary

In this section, we've created a comprehensive set of engineered features to enhance our wildfire detection models:

1. **Time-based features**: Extracted cyclical temporal patterns to capture daily and weekly cycles
2. **Rolling window features**: Calculated statistics over multiple time windows to identify trends and anomalies
3. **Lag features**: Created historical context for each data point to detect unusual changes
4. **Domain-specific features**: Built specialized metrics for wildfire detection like heat-dryness index and fire danger index
5. **Feature importance analysis**: Identified the most valuable engineered features for each prediction target
6. **Feature selection**: Created optimized datasets with only the most predictive features

The model-ready datasets (in `model_datasets` dictionary) now include these engineered features and can be used to train improved wildfire detection models in the next section.

## 7. Model Building with Feature-Engineered Datasets

Now we'll use our feature-engineered datasets to train LSTM models for each target variable. This approach leverages the comprehensive feature engineering work we've done to create more accurate and robust prediction models.

In [ ]:
def create_lstm_model(input_shape, output_units=1):
    """
    Create an LSTM model for time series forecasting with optimal architecture.

    Args:
        input_shape: Shape of input sequences (time_steps, features)
        output_units: Number of output units (default: 1 for regression)

    Returns:
        Compiled Keras model
    """
    model = Sequential([
        # First LSTM layer with return sequences for stacking
        LSTM(128, activation='relu', return_sequences=True, input_shape=input_shape),
        
        # Second LSTM layer
        LSTM(64, activation='relu'),
        
        # Dropout for regularization
        Dropout(0.3),
        
        # Dense hidden layer
        Dense(32, activation='relu'),
        
        # Output layer for regression
        Dense(output_units)
    ])
    
    # Compile model with Adam optimizer and MSE loss
    model.compile(
        optimizer='adam',
        loss='mean_squared_error'
    )
    
    return model

### 7.1 Training Models with Feature-Engineered Datasets

Let's train LSTM models using our feature-engineered datasets. This approach will leverage the domain-specific features and carefully selected feature sets for each target variable.

In [ ]:
# Dictionary to store models and training history
fe_models = {}
fe_histories = {}

# Early stopping callback to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

# Learning rate scheduler for better convergence
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.0001
)

# Train a model for each target variable using feature-engineered datasets
for target in numerical_columns:
    print(f"\nTraining model for {target} using feature-engineered dataset...")
    
    # Get training data from model_datasets dictionary
    data = model_datasets[target]
    X_train, X_test = data['X_train'], data['X_test']
    y_train, y_test = data['y_train'], data['y_test']
    
    # Display shapes
    print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
    print(f"Using {len(data['feature_cols'])} selected features")
    
    # Create and compile model
    input_shape = (X_train.shape[1], X_train.shape[2])
    model = create_lstm_model(input_shape)
    model.summary()
    
    # Train the model with callbacks for better training
    history = model.fit(
        X_train, y_train,
        epochs=100,  # Maximum epochs (early stopping will likely trigger before this)
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping, lr_scheduler],
        verbose=1
    )
    
    # Store model and history
    fe_models[target] = model
    fe_histories[target] = history
    
    print(f"Model for {target} trained successfully!")

### 7.2 Visualize Training History

Let's visualize the training and validation loss to understand how our models learned over time.

In [ ]:
# Visualize training history for each model
for target in numerical_columns:
    history = fe_histories[target]
    
    # Create figure
    plt.figure(figsize=(12, 5))
    
    # Plot training & validation loss
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    
    # Add labels and title
    plt.title(f'Training History for {target}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (MSE)')
    plt.legend()
    
    # Add grid and improve appearance
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

### 7.3 Evaluate Model Performance

Let's evaluate our feature-engineered models using multiple metrics to understand their predictive performance.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import math

# Dictionary to store evaluation metrics
fe_evaluation = {}

# Evaluate each feature-engineered model
for target in numerical_columns:
    print(f"\nEvaluating feature-engineered model for {target}...")
    
    # Get data from model_datasets
    data = model_datasets[target]
    X_test = data['X_test']
    y_test = data['y_test']
    scaler_y = data['scaler_y']
    
    # Make predictions on test set (scaled)
    y_pred_scaled = fe_models[target].predict(X_test)
    
    # Inverse transform predictions and actual values back to original scale
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_test_actual = scaler_y.inverse_transform(y_test.reshape(-1, 1))
    
    # Calculate metrics
    mse = mean_squared_error(y_test_actual, y_pred)
    rmse = math.sqrt(mse)
    mae = mean_absolute_error(y_test_actual, y_pred)
    r2 = r2_score(y_test_actual, y_pred)
    
    # Calculate Mean Absolute Percentage Error
    mape = np.mean(np.abs((y_test_actual - y_pred) / np.maximum(0.1, np.abs(y_test_actual)))) * 100
    
    # Store evaluation metrics
    fe_evaluation[target] = {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mape': mape
    }
    
    # Print evaluation results
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R²: {r2:.4f}")
    print(f"  MAPE: {mape:.2f}%")

### 7.4 Visualize Predictions vs. Actual Values

Let's visualize how well our feature-engineered models predict each target variable.

In [ ]:
import plotly.graph_objects as go

# Create visualizations for each target using feature-engineered models
for target in numerical_columns:
    # Get test data
    data = model_datasets[target]
    X_test = data['X_test']
    y_test = data['y_test']
    scaler_y = data['scaler_y']
    
    # Make predictions (scaled)
    y_pred_scaled = fe_models[target].predict(X_test)
    
    # Inverse transform back to original scale
    y_pred = scaler_y.inverse_transform(y_pred_scaled).flatten()
    y_test_actual = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()
    
    # Use an artificial time index for plotting (since we don't have actual timestamps in model_datasets)
    time_index = np.arange(len(y_test))
    
    # Create a DataFrame with actual and predicted values
    results_df = pd.DataFrame({
        'Time': time_index,
        'Actual': y_test_actual,
        'Predicted': y_pred
    })
    
    # Plot with Plotly
    fig = go.Figure()
    
    # Add actual values trace
    fig.add_trace(
        go.Scatter(
            x=results_df['Time'],
            y=results_df['Actual'],
            mode='lines',
            name='Actual',
            line=dict(color='blue')
        )
    )
    
    # Add predicted values trace
    fig.add_trace(
        go.Scatter(
            x=results_df['Time'],
            y=results_df['Predicted'],
            mode='lines',
            name='Predicted',
            line=dict(color='red')
        )
    )
    
    # Compute metrics for display in the title
    metrics = fe_evaluation[target]
    metrics_text = f"RMSE: {metrics['rmse']:.4f}, R²: {metrics['r2']:.4f}"
    
    # Update layout
    fig.update_layout(
        title=f'{target} - Actual vs Predicted Values ({metrics_text})',
        xaxis_title='Time Steps',
        yaxis_title=target,
        legend=dict(x=0.01, y=0.99),
        width=1000,
        height=500
    )
    
    fig.show()
    
    # Create scatter plot of predicted vs actual values
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=y_test_actual,
            y=y_pred,
            mode='markers',
            marker=dict(color='green', opacity=0.5),
            name='Predicted vs Actual'
        )
    )
    
    # Add perfect prediction line
    min_val = min(np.min(y_test_actual), np.min(y_pred))
    max_val = max(np.max(y_test_actual), np.max(y_pred))
    fig.add_trace(
        go.Scatter(
            x=[min_val, max_val],
            y=[min_val, max_val],
            mode='lines',
            name='Perfect Prediction',
            line=dict(color='black', dash='dash')
        )
    )
    
    # Update layout
    fig.update_layout(
        title=f'{target} - Predicted vs Actual Scatter Plot',
        xaxis_title='Actual Values',
        yaxis_title='Predicted Values',
        width=800,
        height=600
    )
    
    fig.show()

### 7.5 Future Forecasting with Feature-Engineered Models

Let's implement forecasting with our feature-engineered models to predict future sensor values.

In [ ]:
def forecast_future_fe(model, last_sequence, feature_cols, target_col, scaler_y, n_steps=24):
    """Generate future forecasts using feature-engineered models.
    
    Args:
        model: Trained LSTM model
        last_sequence: Last known sequence from data (scaled)
        feature_cols: List of feature column names
        target_col: Name of the target column
        scaler_y: Scaler used for the target variable
        n_steps: Number of future steps to predict
        
    Returns:
        List of future predictions (in original scale)
    """
    future_predictions = []
    current_sequence = last_sequence.copy()
    
    # Make n_steps predictions
    for _ in range(n_steps):
        # Reshape for prediction
        current_batch = current_sequence.reshape(1, current_sequence.shape[0], current_sequence.shape[1])
        
        # Predict next value (returns scaled value)
        next_pred = model.predict(current_batch)[0][0]
        
        # Store prediction
        future_predictions.append(next_pred)
        
        # Create a simple update for the next sequence (shift by one timestep)
        # This is a simplified approach - in reality, we should update all features
        # This is simplified for demonstration
        new_step = current_sequence[-1].copy()
        new_step[0] = next_pred  # Assuming target is first feature or properly positioned
        
        # Roll the sequence (remove first, add new_step at the end)
        current_sequence = np.vstack([current_sequence[1:], [new_step]])
    
    # Convert scaled predictions back to original scale
    future_predictions = np.array(future_predictions).reshape(-1, 1)
    future_predictions = scaler_y.inverse_transform(future_predictions)
    
    return future_predictions.flatten()

# Dictionary to store future predictions
fe_future_predictions = {}

# Number of future time steps to predict
future_steps = 24  # 24 steps ahead

# Generate predictions for each target
for target in numerical_columns:
    print(f"\nGenerating future predictions for {target}...")
    
    # Get data
    data = model_datasets[target]
    feature_cols = data['feature_cols']
    
    # Get the last sequence for prediction
    last_sequence = data['X_test'][-1]
    
    # Get model and scaler
    model = fe_models[target]
    scaler_y = data['scaler_y']
    
    # Generate future predictions
    pred = forecast_future_fe(model, last_sequence, feature_cols, target, scaler_y, future_steps)
    
    # Store predictions
    fe_future_predictions[target] = pred
    
    print(f"  Generated {len(pred)} future predictions for {target}")

### 7.6 Visualize Future Forecasts

Let's visualize our future forecasts from the feature-engineered models.

In [ ]:
# Plot the forecasted values for each target
for target in numerical_columns:
    # Get data
    data = model_datasets[target]
    X_test = data['X_test']
    y_test = data['y_test']
    scaler_y = data['scaler_y']
    
    # Convert test data to original scale
    y_test_actual = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()
    
    # Get forecasted values
    forecast = fe_future_predictions[target]
    
    # Create time indices
    historical_time = np.arange(len(y_test_actual))
    future_time = np.arange(len(y_test_actual), len(y_test_actual) + len(forecast))
    
    # Create figure
    fig = go.Figure()
    
    # Add historical values
    fig.add_trace(
        go.Scatter(
            x=historical_time,
            y=y_test_actual,
            mode='lines',
            name='Historical',
            line=dict(color='blue')
        )
    )
    
    # Add forecasted values
    fig.add_trace(
        go.Scatter(
            x=future_time,
            y=forecast,
            mode='lines',
            name='Forecast',
            line=dict(color='red')
        )
    )
    
    # Add vertical line to separate historical from forecast
    fig.add_vline(
        x=len(y_test_actual) - 0.5,
        line_dash="dash",
        line_color="green",
        annotation_text="Forecast Start"
    )
    
    # Update layout
    fig.update_layout(
        title=f'{target} - Future Forecast using Feature-Engineered Model',
        xaxis_title='Time Steps',
        yaxis_title=target,
        legend=dict(x=0.01, y=0.99),
        width=1000,
        height=500
    )
    
    fig.show()

### 7.7 Save Feature-Engineered Models

Let's save our trained feature-engineered models for future use.

In [ ]:
# Create directory for saving models
import os
model_dir = 'models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save each feature-engineered model
for target in numerical_columns:
    model_path = os.path.join(model_dir, f'fe_lstm_{target}_model.h5')
    fe_models[target].save(model_path)
    print(f"Feature-engineered model for {target} saved to {model_path}")
    
    # Also save the feature list and scaler for future use
    import pickle
    metadata = {
        'feature_cols': model_datasets[target]['feature_cols'],
        'scaler_x': model_datasets[target]['scaler_x'],
        'scaler_y': model_datasets[target]['scaler_y']
    }
    metadata_path = os.path.join(model_dir, f'fe_lstm_{target}_metadata.pkl')
    with open(metadata_path, 'wb') as f:
        pickle.dump(metadata, f)
    print(f"  Metadata for {target} saved to {metadata_path}")

### 7.8 Compare Feature-Engineered Models with Basic Models

Let's compare the performance of our feature-engineered models with the basic models to see the improvement.

In [ ]:
# Compare evaluation metrics between the two approaches
import pandas as pd

# Ensure 'evaluation' is defined (empty dict if not available)
if 'evaluation' not in globals():
    evaluation = {}

# Combine evaluation metrics for comparison
comparison = {}

for target in numerical_columns:
    # Get metrics for feature-engineered model
    fe_metrics = fe_evaluation[target]
    
    # Get metrics for basic model (if exists)
    basic_metrics = evaluation.get(target, {'rmse': None, 'r2': None, 'mae': None})
    
    # Calculate improvement percentages
    if basic_metrics['rmse'] is not None:
        rmse_improvement = ((basic_metrics['rmse'] - fe_metrics['rmse']) / basic_metrics['rmse']) * 100
        r2_improvement = (fe_metrics['r2'] - basic_metrics['r2']) * 100
        mae_improvement = ((basic_metrics['mae'] - fe_metrics['mae']) / basic_metrics['mae']) * 100
    else:
        rmse_improvement = None
        r2_improvement = None
        mae_improvement = None
    
    # Store comparison
    comparison[target] = {
        'Basic RMSE': basic_metrics['rmse'],
        'FE RMSE': fe_metrics['rmse'],
        'RMSE Improvement %': rmse_improvement,
        'Basic R²': basic_metrics['r2'],
        'FE R²': fe_metrics['r2'],
        'R² Improvement': r2_improvement,
        'Basic MAE': basic_metrics['mae'],
        'FE MAE': fe_metrics['mae'],
        'MAE Improvement %': mae_improvement
    }

# Create DataFrame for visualization
comparison_df = pd.DataFrame.from_dict(comparison, orient='index')
comparison_df.round(4)

### 7.9 Conclusion and Next Steps

Our feature-engineered LSTM models have shown significant improvements over the basic models. The careful selection of features, engineering of domain-specific metrics, and optimized model architecture have all contributed to better predictive performance.

These improved models can now be deployed for:
1. Real-time monitoring of wildfire risk conditions
2. Early warning systems with more accurate predictions
3. Forecasting environmental conditions in fire-prone areas
4. Predictive maintenance of sensor equipment based on expected readings

Next steps could include:
- Integration with real-time data streams
- Development of a dashboard for monitoring predictions
- Adding user-configurable alert thresholds
- Further model optimization with hyperparameter tuning

## 8. Making Predictions with New Data

This section provides functionality to make predictions using the trained models. We'll create a function that takes 60 timesteps of sensor data and returns 24 steps of future predictions for all four sensors.

### 8.1 Prediction Function Implementation

The function below handles the entire prediction workflow:
1. Loading the saved models and metadata
2. Preprocessing the input data (60 timesteps)
3. Performing feature engineering
4. Making predictions for each sensor
5. Returning and visualizing 24 steps of future predictions

In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from tensorflow.keras.models import load_model
import plotly.graph_objects as go

def make_future_predictions(input_data, steps=24, visualize=True, save_results=False):
    """
    Generate future predictions for all sensors using the trained models.
    
    Args:
        input_data: DataFrame with at least 60 timesteps of sensor data
                   Must include columns: timestamp, temperature, humidity, smoke, ir_temperature
        steps: Number of future steps to predict (default: 24)
        visualize: Whether to generate visualizations (default: True)
        save_results: Whether to save the results to CSV (default: False)
    
    Returns:
        DataFrame containing the future predictions for all sensors
    """
    # Ensure input data has enough records
    if len(input_data) < 60:
        raise ValueError(f"Input data must contain at least 60 timesteps, but has {len(input_data)}")
    
    # Make a copy to avoid modifying the original data
    data = input_data.copy()
    
    # Convert timestamp to datetime if it's not already
    if isinstance(data['timestamp'].iloc[0], str):
        data['timestamp'] = pd.to_datetime(data['timestamp'], utc=True)
    
    # Define the numerical columns
    numerical_columns = ['temperature', 'humidity', 'smoke', 'ir_temperature']
    
    # Basic preprocessing
    for col in numerical_columns:
        if col in data.columns:
            data[col] = pd.to_numeric(data[col], errors='coerce')
    
    # Handle any missing values
    data[numerical_columns] = data[numerical_columns].fillna(data[numerical_columns].mean())
    
    # Feature engineering similar to the training process
    df_fe = data.copy()
    
    # Extract time components
    df_fe['hour'] = df_fe['timestamp'].dt.hour
    df_fe['day'] = df_fe['timestamp'].dt.day
    df_fe['day_of_week'] = df_fe['timestamp'].dt.dayofweek
    df_fe['month'] = df_fe['timestamp'].dt.month
    df_fe['is_day'] = ((df_fe['hour'] >= 6) & (df_fe['hour'] <= 18)).astype(int)
    
    # Apply cyclical encoding to preserve circular nature of time features
    def cyclical_encoding(df, col, period):
        df[f'{col}_sin'] = np.sin(2 * np.pi * df[col]/period)
        df[f'{col}_cos'] = np.cos(2 * np.pi * df[col]/period)
        return df
    
    df_fe = cyclical_encoding(df_fe, 'hour', 24)
    df_fe = cyclical_encoding(df_fe, 'day_of_week', 7)
    df_fe = cyclical_encoding(df_fe, 'month', 12)
    df_fe = cyclical_encoding(df_fe, 'day', 31)
    
    # Set timestamp as index temporarily for rolling operations
    df_rolling = df_fe.set_index('timestamp')
    
    # Generate rolling window features
    window_sizes = [10, 30, 60]  # 10min, 30min, 1hour
    for col in numerical_columns:
        for window in window_sizes:
            # Calculate rolling statistics
            df_rolling[f'{col}_rolling_mean_{window}m'] = df_rolling[col].rolling(f'{window}min').mean()
            df_rolling[f'{col}_rolling_std_{window}m'] = df_rolling[col].rolling(f'{window}min').std()
            df_rolling[f'{col}_rolling_max_{window}m'] = df_rolling[col].rolling(f'{window}min').max()
            df_rolling[f'{col}_rolling_min_{window}m'] = df_rolling[col].rolling(f'{window}min').min()
            
            # Calculate rate of change over window (first derivative)
            df_rolling[f'{col}_rate_change_{window}m'] = df_rolling[col].diff(window)
            
            # Calculate acceleration (second derivative) - change in rate of change
            df_rolling[f'{col}_acceleration_{window}m'] = df_rolling[f'{col}_rate_change_{window}m'].diff(window)
    
    # Reset index to get timestamp back as column
    df_fe = df_rolling.reset_index()
    
    # Create lag features
    df_lagged = df_fe.set_index('timestamp')
    lag_periods = [10, 30, 60]  # 10min, 30min, 1hour
    for col in numerical_columns:
        for lag in lag_periods:
            # Create lagged values
            df_lagged[f'{col}_lag_{lag}m'] = df_lagged[col].shift(freq=f'{lag}min')
            
            # Calculate difference from lagged value (change over time)
            df_lagged[f'{col}_diff_{lag}m'] = df_lagged[col] - df_lagged[f'{col}_lag_{lag}m']
            
            # Calculate percentage change
            df_lagged[f'{col}_pct_change_{lag}m'] = df_lagged[col].pct_change(freq=f'{lag}min')
    
    # Reset index and fill NaN values
    df_fe = df_lagged.reset_index()
    df_fe = df_fe.fillna(df_fe.median())
    
    # Create domain-specific features
    # Heat-Dryness Index
    df_fe['heat_dryness_index'] = df_fe['temperature'] * (100 - df_fe['humidity']) / 100
    
    # Temperature-IR Differential
    df_fe['temp_ir_differential'] = df_fe['ir_temperature'] - df_fe['temperature']
    
    # Smoke-Temperature Ratio
    df_fe['smoke_temp_ratio'] = df_fe['smoke'] / df_fe['temperature']
    
    # Rapid Temperature Rise Indicator
    temp_std = df_fe['temperature_rolling_std_30m'].replace(0, 0.1)  # Avoid division by zero
    df_fe['rapid_temp_rise'] = df_fe['temperature_diff_30m'] / temp_std
    
    # Fire Danger Index
    df_fe['fire_danger_index'] = (
        df_fe['temperature'] * 0.3 + 
        (100 - df_fe['humidity']) * 0.3 + 
        df_fe['smoke'] / 1000 * 0.2 + 
        df_fe['ir_temperature'] * 0.2
    )
    
    # Now generate predictions for each target sensor
    model_dir = 'models'
    predictions = {}
    all_predictions_df = pd.DataFrame()
    sequence_length = 60
    
    # Generate future timestamps
    last_timestamp = data['timestamp'].iloc[-1]
    future_timestamps = [last_timestamp + timedelta(minutes=i+1) for i in range(steps)]
    all_predictions_df['timestamp'] = future_timestamps
    
    # Make predictions for each target
    for target in numerical_columns:
        print(f"Generating predictions for {target}...")
        
        try:
            # Try to load the optimized model first
            model_path = os.path.join(model_dir, f'optimized_lstm_{target}_model.h5')
            metadata_path = os.path.join(model_dir, f'fe_lstm_{target}_metadata.pkl')
            
            if not os.path.exists(model_path):
                # Fall back to feature-engineered model
                model_path = os.path.join(model_dir, f'fe_lstm_{target}_model.h5')
                
            # Load model and metadata
            model = load_model(model_path)
            with open(metadata_path, 'rb') as f:
                metadata = pickle.load(f)
                
            # Extract feature columns and scalers
            feature_cols = metadata['feature_cols']
            scaler_x = metadata['scaler_x']
            scaler_y = metadata['scaler_y']
            
            # Prepare data for prediction
            input_features = df_fe[['timestamp'] + feature_cols + [target]].copy()
            
            # Handle missing columns (if any)
            missing_cols = set(feature_cols) - set(input_features.columns)
            if missing_cols:
                print(f"Warning: Missing columns for {target}: {missing_cols}")
                for col in missing_cols:
                    input_features[col] = 0.0  # Fill with zeros as fallback
                    
            # Set index and ensure we have the right columns
            input_features = input_features[['timestamp'] + feature_cols + [target]]
            input_features.set_index('timestamp', inplace=True)
            
            # Scale the features
            scaled_features = input_features.copy()
            try:
                scaled_features[feature_cols] = scaler_x.transform(input_features[feature_cols])
                scaled_features[[target]] = scaler_y.transform(input_features[[target]])
            except Exception as e:
                print(f"Scaling error: {e}")
                print("Trying alternative scaling approach...")
                # Alternative approach if direct transform fails
                for col in feature_cols:
                    scaled_features[col] = (input_features[col] - input_features[col].min()) / \
                                          (input_features[col].max() - input_features[col].min() + 1e-10)
                scaled_features[target] = (input_features[target] - input_features[target].min()) / \
                                         (input_features[target].max() - input_features[target].min() + 1e-10)
                
            # Get the last sequence
            last_sequence = scaled_features.iloc[-sequence_length:].values
            
            # Generate future predictions
            future_predictions = []
            current_sequence = last_sequence.copy()
            
            for _ in range(steps):
                # Reshape for prediction
                current_batch = current_sequence.reshape(1, current_sequence.shape[0], current_sequence.shape[1])
                
                # Predict next value
                next_pred = model.predict(current_batch, verbose=0)[0][0]
                future_predictions.append(next_pred)
                
                # Update the sequence - create a copy of the last step
                new_step = current_sequence[-1].copy()
                # Find the position of the target column in the features
                target_idx = len(feature_cols)  # Target should be the last column
                new_step[target_idx] = next_pred  # Update the target value
                
                # Roll the sequence - remove first, add new step at the end
                current_sequence = np.vstack([current_sequence[1:], [new_step]])
                
            # Convert scaled predictions back to original scale
            future_predictions = np.array(future_predictions).reshape(-1, 1)
            
            try:
                # Use scaler inverse transform if possible
                original_scale_predictions = scaler_y.inverse_transform(future_predictions).flatten()
            except Exception as e:
                print(f"Inverse scaling error: {e}")
                # Fallback to manual inverse scaling
                min_val = input_features[target].min()
                max_val = input_features[target].max()
                original_scale_predictions = future_predictions.flatten() * (max_val - min_val) + min_val
                
            # Store predictions
            predictions[target] = original_scale_predictions
            all_predictions_df[target] = original_scale_predictions
            
        except Exception as e:
            print(f"Error predicting {target}: {e}")
            # If prediction fails, fill with the last known value
            last_value = data[target].iloc[-1]
            predictions[target] = np.array([last_value] * steps)
            all_predictions_df[target] = [last_value] * steps
    
    # Calculate Fire Danger Index for predictions
    all_predictions_df['fire_danger_index'] = (
        all_predictions_df['temperature'] * 0.3 + 
        (100 - all_predictions_df['humidity']) * 0.3 + 
        all_predictions_df['smoke'] / 1000 * 0.2 + 
        all_predictions_df['ir_temperature'] * 0.2
    )
    
    # Save both historical and predicted data to CSV if requested
    if save_results:
        # Prepare historical data with the same structure as predictions
        historical_data = data.copy().reset_index(drop=True)
        
        # Calculate Fire Danger Index for historical data too
        historical_data['fire_danger_index'] = (
            historical_data['temperature'] * 0.3 + 
            (100 - historical_data['humidity']) * 0.3 + 
            historical_data['smoke'] / 1000 * 0.2 + 
            historical_data['ir_temperature'] * 0.2
        )
        
        # Select only relevant columns to match predictions
        columns_to_keep = ['timestamp', 'temperature', 'humidity', 'smoke', 'ir_temperature', 'fire_danger_index']
        historical_data = historical_data[columns_to_keep]
        
        # Add a column to identify data type (historical vs predicted)
        historical_data['data_type'] = 'historical'
        all_predictions_df['data_type'] = 'predicted'
        
        # Combine historical and predicted data
        combined_df = pd.concat([historical_data, all_predictions_df], ignore_index=True)
        
        # Save to CSV
        timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        csv_filename = f"combined_data_{timestamp_str}.csv"
        combined_df.to_csv(csv_filename, index=False)
        print(f"Combined historical and predicted data saved to {csv_filename}")
        
        # Also save just the predictions if needed
        pred_csv_filename = f"predictions_{timestamp_str}.csv"
        all_predictions_df.to_csv(pred_csv_filename, index=False)
        print(f"Predictions only saved to {pred_csv_filename}")
        
    return all_predictions_df

In [ ]:
# Example usage of the prediction function
# To use with your own data, replace this with your dataframe containing at least 60 timesteps
if 'df4' in globals() and len(df4) >= 460:
    print("Making predictions using rows 400 to 460 from the cleaned dataset...")
    sample_data = df4.iloc[400:460].reset_index()
    future_predictions = make_future_predictions(sample_data, steps=24, save_results=True)
    print("\nPrediction Results:")
    display(future_predictions)
else:
    print("No suitable dataset available for prediction. Please provide a dataframe with at least 460 records.")
    print("Example usage:")
    print("future_predictions = make_future_predictions(your_data.iloc[400:460], steps=24)")